# Phi-4 Mini ONNX Παράλληλη Κλήση Συναρτήσεων - Οδηγός

Αυτό το σημειωματάριο δείχνει πώς να χρησιμοποιήσετε το Phi-4 Mini με το ONNX Runtime GenAI για παράλληλη κλήση συναρτήσεων. Η κλήση συναρτήσεων επιτρέπει στο μοντέλο να καλεί έξυπνα εξωτερικά εργαλεία και APIs βάσει αιτημάτων του χρήστη.

## Επισκόπηση

Σε αυτόν τον οδηγό, θα μάθετε πώς να:
- Ρυθμίσετε το Phi-4 Mini με το ONNX Runtime GenAI
- Ορίσετε σχήματα συναρτήσεων για κράτηση πτήσεων και ξενοδοχείων
- Χρησιμοποιήσετε καθοδηγούμενη παραγωγή με γραμματική Lark για δομημένη έξοδο
- Εκτελέσετε παράλληλες κλήσεις συναρτήσεων για σύνθετα σενάρια κράτησης ταξιδιών

## Προαπαιτούμενα

Πριν εκτελέσετε αυτό το σημειωματάριο, βεβαιωθείτε ότι έχετε:
- Κατεβάσει το μοντέλο Phi-4 Mini ONNX
- Εγκαταστήσει το πακέτο `onnxruntime-genai`
- Βασική κατανόηση των εννοιών κλήσης συναρτήσεων


## Βήμα 1: Εισαγωγή Απαραίτητων Βιβλιοθηκών

Αρχικά, θα εισάγουμε τις απαραίτητες βιβλιοθήκες για την υλοποίηση της κλήσης της συνάρτησης.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Βήμα 2: Ρύθμιση και Διαμόρφωση Μοντέλου

Τώρα θα διαμορφώσουμε το μοντέλο Phi-4 Mini ONNX. Βεβαιωθείτε ότι αντικαθιστάτε τη διαδρομή με τον πραγματικό κατάλογο του μοντέλου σας.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Βήμα 3: Ρύθμιση Παραμέτρων Γεννήτριας

Ρυθμίστε τις παραμέτρους γεννήτριας για να ελέγξετε τη συμπεριφορά εξόδου του μοντέλου. Αυτές οι ρυθμίσεις εξασφαλίζουν καθορισμένες και εστιασμένες απαντήσεις για την κλήση λειτουργιών.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Βήμα 4: Ορισμός Διαθέσιμων Λειτουργιών

Εδώ ορίζουμε τις λειτουργίες που μπορεί να καλέσει ο βοηθός AI. Σε αυτό το παράδειγμα, έχουμε δύο λειτουργίες που σχετίζονται με ταξίδια:
1. **booking_flight_tickets**: Για κράτηση πτήσεων μεταξύ αεροδρομίων
2. **booking_hotels**: Για κράτηση καταλυμάτων σε ξενοδοχεία

Οι ορισμοί των λειτουργιών ακολουθούν τη μορφή σχήματος κλήσης λειτουργιών της OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Βήμα 5: Βοηθητικές Συναρτήσεις Δημιουργίας Γραμματικής

Αυτές οι βοηθητικές συναρτήσεις μετατρέπουν τους ορισμούς των συναρτήσεών μας σε μορφή γραμματικής Lark, η οποία χρησιμοποιείται από το ONNX Runtime GenAI για καθοδηγούμενη δημιουργία. Αυτό διασφαλίζει ότι το μοντέλο παράγει έγκυρες κλήσεις συναρτήσεων στη σωστή μορφή JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Βήμα 6: Δοκιμή Δημιουργίας Γραμματικής

Ας δοκιμάσουμε τις λειτουργίες δημιουργίας γραμματικής για να δούμε πώς μετατρέπουν τους ορισμούς των εργαλείων μας στη σωστή μορφή.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Βήμα 7: Προετοιμασία της Προτροπής Συστήματος και του Γεννήτορα

Τώρα θα δημιουργήσουμε την προτροπή συστήματος που ενημερώνει το μοντέλο για τα διαθέσιμα εργαλεία και θα ρυθμίσουμε τον γεννήτορα με καθοδηγούμενες παραμέτρους δημιουργίας.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Βήμα 8: Αρχικοποίηση Γεννήτριας με Καθοδηγούμενη Δημιουργία

Τώρα θα δημιουργήσουμε τη γεννήτρια με τις παραμετροποιήσεις μας και θα εφαρμόσουμε τη γραμματική Lark για καθοδηγούμενη δημιουργία.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Βήμα 9: Δοκιμή Παράλληλης Κλήσης Συναρτήσεων

Τώρα ας δοκιμάσουμε τη ρύθμισή μας με ένα σύνθετο σενάριο κράτησης ταξιδιού που απαιτεί την κλήση πολλαπλών συναρτήσεων.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Δημιουργία Κλήσεων Συναρτήσεων

Το μοντέλο θα δημιουργήσει τώρα δομημένες κλήσεις συναρτήσεων βάσει του αιτήματος του χρήστη. Χάρη στη καθοδηγούμενη δημιουργία, η έξοδος θα είναι σε έγκυρη μορφή JSON που μπορεί να εκτελεστεί άμεσα.

**Αναμενόμενη Έξοδος**: Το μοντέλο θα πρέπει να δημιουργήσει κλήσεις συναρτήσεων για:
1. `booking_flight_tickets` με λεπτομέρειες από Πεκίνο (PEK) προς Παρίσι (CDG)
2. `booking_hotels` με λεπτομέρειες διαμονής στο Παρίσι

Εκτελέστε το παρακάτω κελί για να δείτε τη ζωντανή δημιουργία:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Συμπέρασμα

🎉 **Συγχαρητήρια!** Έχετε υλοποιήσει με επιτυχία την παράλληλη κλήση συναρτήσεων με το Phi-4 Mini χρησιμοποιώντας το ONNX Runtime GenAI.

### Τι Έχετε Μάθει:

1. **Ρύθμιση Μοντέλου**: Πώς να διαμορφώσετε το Phi-4 Mini με το ONNX Runtime GenAI
2. **Ορισμός Συναρτήσεων**: Πώς να ορίσετε σχήματα συναρτήσεων για κλήσεις AI συναρτήσεων
3. **Καθοδηγούμενη Δημιουργία**: Πώς να χρησιμοποιήσετε τη γραμματική Lark για δομημένη παραγωγή αποτελεσμάτων
4. **Παράλληλες Κλήσεις Συναρτήσεων**: Πώς να διαχειριστείτε σύνθετα αιτήματα που απαιτούν πολλαπλές κλήσεις συναρτήσεων

### Βασικά Οφέλη:

- ✅ **Δομημένη Παραγωγή**: Η καθοδηγούμενη δημιουργία εξασφαλίζει έγκυρες κλήσεις συναρτήσεων σε JSON
- ✅ **Παράλληλη Επεξεργασία**: Διαχείριση πολλαπλών κλήσεων συναρτήσεων σε ένα μόνο αίτημα
- ✅ **Υψηλή Απόδοση**: Το ONNX Runtime παρέχει βελτιστοποιημένη εξαγωγή αποτελεσμάτων
- ✅ **Ευέλικτο Σχήμα**: Εύκολη προσθήκη ή τροποποίηση ορισμών συναρτήσεων

### Πόροι:

- [Τεκμηρίωση Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Τεκμηρίωση ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Βέλτιστες Πρακτικές Κλήσεων Συναρτήσεων](https://platform.openai.com/docs/guides/function-calling)



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
